# Spelling Correction

> *Text Analytics*  
> *MSc in Data Science, Department of Informatics*  
> *Athens University of Economics and Business*

---


i) Implement a bigram and a trigram language model for sentences, using Laplace smoothing. In practice, n-gram language models compute the sum of the logarithms of the n-gram probabilities of each sequence, instead of their product and you should do the same. Assume that each sentence starts with the pseudo-token *start* (or two pseudo-tokens *start1*, *start2* for the trigram model) and ends with the pseudo-token *end*. Train your models on a training subset of a corpus (e.g., a subset of a corpus included in NLTK – see http://www.nltk.org/). Include in the vocabulary only words that occur, e.g., at least 10 times in the training subset. Use the same vocabulary in the bigram and trigram models. Replace all out-of-vocabulary (OOV) words (in the training, development, test subsets) by a special token *UNK*. Alternatively, you may want to use BPEs instead of words (obtaining the BPE vocabulary from your training subset) to avoid unknown words. 

See Section 2.4.3 (“Byte-Pair Encoding for Tokenization”) of the 3rd edition of Jurafsky & Martin’s book (https://web.stanford.edu/~jurafsky/slp3/).

For more information, check https://huggingface.co/transformers/master/tokenizer_summary.html.

ii) Estimate the language cross-entropy and perplexity of your two models on a test subset of the corpus, treating the entire test subset as a single sequence of sentences, with *start* (or *start1*, *start2*) at the beginning of each sentence, and *end* at the end of each sentence. Do not include probabilities of the form P(*start*|…) or P(*start1*|…), P(*start2*|…) in the computation of cross-entropy and perplexity, since we are not predicting the start pseudo-tokens; but include probabilities of the form P(*end*|…), since we do want to be able to predict if a word will be the last one of a sentence. You must also count *end* tokens (but not *start*, *start1*, *start2* tokens) in the total length N of the test corpus.

iii) Develop a context-aware spelling corrector using your bigram language model and a beam search decoder. If you are keen, you can also try using your trigram model. You can use the inverse of the Levenshtein distance between 𝑤𝑖,𝑡𝑖 as 𝑃(𝑤𝑖|𝑡𝑖).


## Libraries 

In [1]:
from nltk.corpus import stopwords
from nltk.corpus import treebank
from nltk import ngrams
from nltk import edit_distance
from collections import Counter, defaultdict
from wordcloud import WordCloud
from random import sample
from operator import itemgetter
import nltk
import math
import string
import math
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#nltk.download('treebank')
#nltk.download('stopwords')

## Data

For the purpose of this exercise we are going to use **The Penn Treebank** dataset, which is a diverse and extensive collection of text genres, encompassing news articles, fiction, and academic writings, totaling over 4.5 million words. Annotated with part-of-speech tags, syntactic structures, and named entities, it provides a rich resource for natural language processing tasks. Notably, the dataset includes detailed sentence structures, offering phrase structure trees that are invaluable for tasks involving syntax and grammar. Widely utilized in research, the Penn Treebank has significantly influenced the development of natural language processing algorithms and serves as a benchmark dataset for evaluating models across various linguistic domains.

##### *Function to process the dataset and create the vocabulary*

In [2]:
def get_dataset_corpus_vocabulary(dataset,stopwords):
    
    # Get words from the dataset
    dataset_words = dataset.words()

    # Get words frequency
    fdist = nltk.FreqDist(dataset_words)

    # Keep words with frequency >=10
    vocabulary = list(filter(lambda x:x[1]>=10,fdist.items()))

    # map all out-of-vocabulary wordws to *UNK*
    mapping = defaultdict(lambda: 'UNK')

    for word in vocabulary:
        mapping[word[0]] = word[0]
    
    #create corpus
    corpus = []
    for sentence in dataset.sents():
        corpus.append([mapping[word] for word in sentence])
    
    # remove stopwords from vocabulary
    vocabulary = [w for w in vocabulary if not w in stopwords]
    
    return corpus,vocabulary

In [3]:
# create stopword list and extend it to include puncuation marks and more common words
stopwords_list = [stopwords.words('english')]
stopwords_list.extend(string.punctuation)
stopwords_list.extend(["the","of","and","to","The","for","by","*-1","in","0","is","*U*","*T*-1","a","'s","as","*-2","on","it","*T*-2","at","with","from","``","''","that","Mr.","are","its","n't","have","was","be","an","has","--","will","said","or","he","this"])

In [4]:
#execute
corpus, vocabulary = get_dataset_corpus_vocabulary(treebank,stopwords_list)

#split to training - development and test set
training = corpus[0:round(0.6*len(corpus))]
development = corpus[round(0.6*len(corpus)):round(0.8*len(corpus))]
testing = corpus[round(0.8*len(corpus)):]

## Question 1

##### *Functions for n-gram counters*

In [5]:
def calculate_unigram_counter(sentences):
    unigram_counter = Counter()

    for sent in sentences:
        # Update the unigram counter
        unigram_counter.update([(gram,) for gram in ["<s>"] + sent])
    
    return unigram_counter

def calculate_bigram_counter(sentences):
    bigram_counter = Counter()
    
    for sent in sentences:
        # Update the bigram counter
        bigram_pad_sent = ["<s>"] + sent +  ['<e>']    
        bigram_counter.update([(gram1, gram2) for gram1, gram2 in zip(bigram_pad_sent, bigram_pad_sent[1:])])
   
    return bigram_counter 

def calculate_trigram_counter(sentences):
    trigram_counter = Counter()
    
    for sent in sentences:
        # Update the trigram counter
        trigram_pad_sent = ["<s>"]*2 + sent +  ['<e>']*2
        trigram_counter.update([(gram1, gram2, gram3) for gram1, gram2, gram3 in zip(trigram_pad_sent, trigram_pad_sent[1:], trigram_pad_sent[2:])]) 
    
    return trigram_counter

##### *Function to calculate cross-entropy and perplexity*

- **Cross-Entropy**

$H_{\alpha}(P, Q) = -\frac{1}{N} \sum_{i=1}^{N} \log_2\left(\frac{{\text{{count}}(x_i, x_{i-1}) + \alpha}}{{\text{{count}}(x_{i-1}) + \alpha \cdot \text{{vocab\_size}}}}\right)$


- **Perplexity**

$\text{PP}(P, Q) = 2^{H(P, Q)}$

In [6]:
def calculate_crossentropy_perplexity(unigram_counter, bigram_counter, trigram_counter, vocab_size, sentences, alpha, lm):
    # Initialize counters and variables based on the chosen language model (unigram/bigram/trigram)
    if lm == "bigram":
        sum_prob = 0
        bigram_cnt = 0
        for sentence in sentences:
            sent = ['<start>'] + sentence + ['<end>']
            # Iterate over the bigrams of the sentence
            for idx in range(1, len(sent)):
                # Calculate the probability of the bigram using add-alpha smoothing
                bigram_prob = (bigram_counter[(sent[idx - 1], sent[idx])] + alpha) / (
                        unigram_counter[(sent[idx - 1],)] + alpha * vocab_size)
                sum_prob += math.log2(bigram_prob)
                bigram_cnt += 1
        # Calculate cross-entropy and perplexity
        cross_entropy = -sum_prob / bigram_cnt
        perplexity = math.pow(2, cross_entropy)
    else:
        sum_prob = 0
        trigram_cnt = 0
        for sentence in sentences:
            sent = ['<start>'] + ['<start>'] + sentence + ['<end>']
            # Iterate over the trigrams of the sentence
            for idx in range(2, len(sent) - 1):
                # Calculate the probability of the trigram using add-alpha smoothing
                trigram_prob = (trigram_counter[(sent[idx - 2], sent[idx - 1], sent[idx])] + alpha) / (
                        bigram_counter[(sent[idx - 2], sent[idx - 1])] + alpha * vocab_size)
                sum_prob += math.log2(trigram_prob)
                trigram_cnt += 1
        # Calculate cross-entropy and perplexity
        cross_entropy = -sum_prob / trigram_cnt
        perplexity = math.pow(2, cross_entropy)

    return cross_entropy, perplexity

In [7]:
def tune_alpha(unigram_counter, bigram_counter, trigram_counter, vocab_size, sentences, lm):
    # Generate a range of alpha values
    alphas = np.linspace(0.001, 0.1, 100)
    
    # Calculate cross-entropy and perplexity for each alpha
    values = [calculate_crossentropy_perplexity(unigram_counter, bigram_counter, trigram_counter, vocab_size, sentences, a, lm) for a in alphas]
    
    # Separate cross-entropy and perplexity values
    crossentropy = list(map(itemgetter(0), values))
    perplexity = list(map(itemgetter(1), values))
    
    # Find the minimum cross-entropy and its corresponding alpha
    min_crossentropy = min(crossentropy)
    index_min_crossentropy = crossentropy.index(min_crossentropy)
    min_alpha = alphas[index_min_crossentropy]
    
    # Find the minimum perplexity
    min_perplexity = min(perplexity)
    
    return min_crossentropy, min_perplexity, min_alpha

In [8]:
#calculate counters from the training set
unigram_counter = calculate_unigram_counter(training)
bigram_counter = calculate_bigram_counter(training)
trigram_counter = calculate_trigram_counter(training)
vocab_size = len(vocabulary)

##### *Find Optimal Alpha for the bigram and trigram language models*

In [9]:
# best alpha for bigram
crossentropy_bigram, perplexity_bigram, alpha_bigram = tune_alpha(unigram_counter, bigram_counter, trigram_counter, vocab_size, development,"bigram")

# best alpha for trigram
crossentropy_trigram,perplexity_trigram,alpha_trigram = tune_alpha(unigram_counter, bigram_counter, trigram_counter, vocab_size, development,"trigram")

print('='*50, ' Bigram ', '='*50)
print("Cross Entropy: {0:.3f}".format(crossentropy_bigram))
print("Perplexity: {0:.3f}".format(perplexity_bigram))
print(f"The optimal cross entropy for Bigram, {round(crossentropy_bigram, 4)} is achieved for alpha equal to: {round(alpha_bigram, 4)}")

print('\n')

print('='*50, ' Trigram ', '='*50)
print("Cross Entropy: {0:.3f}".format(crossentropy_trigram))
print("Perplexity: {0:.3f}".format(perplexity_trigram))
print(f"The optimal cross entropy for Bigram, {round(crossentropy_trigram, 4)} is achieved for alpha equal to: {round(alpha_trigram, 4)}")

==================================================  Bigram  ==================================================
Cross Entropy: 7.135
Perplexity: 140.579
The optimal cross entropy for Bigram, 7.1352 is achieved for alpha equal to: 0.035


==================================================  Trigram  ==================================================
Cross Entropy: 8.167
Perplexity: 287.434
The optimal cross entropy for Bigram, 8.1671 is achieved for alpha equal to: 0.012


##### *Calculate Cross-Entropy and Perplexity for the training set (tuned alpha)*

In [10]:
# bigram
crossentropy_training_bigram,perplexity_training_bigram = calculate_crossentropy_perplexity(unigram_counter, bigram_counter, trigram_counter, vocab_size, training,alpha_bigram,'bigram')

#trigram
crossentropy_training_trigram,perplexity_training_trigram = calculate_crossentropy_perplexity(unigram_counter, bigram_counter, trigram_counter, vocab_size, training,alpha_trigram,'trigram')

print('='*50, ' Bigram ', '='*50)
print("Cross Entropy in Training set: {0:.3f}".format(crossentropy_training_bigram))
print("Perplexity in Training set: {0:.3f}".format(perplexity_training_bigram))

print('\n')

print('='*50, ' Trigram ', '='*50)
print("Cross Entropy in Training set: {0:.3f}".format(crossentropy_training_trigram))
print("Perplexity in Training set: {0:.3f}".format(perplexity_training_trigram))

==================================================  Bigram  ==================================================
Cross Entropy in Training set: 5.694
Perplexity in Training set: 51.776


==================================================  Trigram  ==================================================
Cross Entropy in Training set: 4.578
Perplexity in Training set: 23.881


## Question 2

##### *Calculate Cross-Entropy and Perplexity for the test set (tuned alpha)*

In [11]:
# bigram
crossentropy_testing_bigram,perplexity_testing_bigram = calculate_crossentropy_perplexity(unigram_counter, bigram_counter, trigram_counter, vocab_size, testing,alpha_bigram,'bigram')

#trigram
crossentropy_testing_trigram,perplexity_testing_trigram = calculate_crossentropy_perplexity(unigram_counter, bigram_counter, trigram_counter, vocab_size, testing,alpha_trigram,'trigram')

print('='*50, ' Bigram ', '='*50)
print("Cross Entropy in Training set: {0:.3f}".format(crossentropy_testing_bigram))
print("Perplexity in Training set: {0:.3f}".format(perplexity_testing_bigram))

print('\n')

print('='*50, ' Trigram ', '='*50)
print("Cross Entropy in Training set: {0:.3f}".format(crossentropy_testing_trigram))
print("Perplexity in Training set: {0:.3f}".format(perplexity_testing_trigram))

==================================================  Bigram  ==================================================
Cross Entropy in Training set: 6.978
Perplexity in Training set: 126.096


==================================================  Trigram  ==================================================
Cross Entropy in Training set: 7.846
Perplexity in Training set: 230.126


## Question 3

##### *Function to calculate the normalized edit distance*

In [15]:
def calculate_normalized_edit_distance(sentence, vocab):
    # Create an array to store probabilities, initialized with infinity
    probability_array = np.full((len(sentence.split()), len(vocab)), np.inf)

    # Iterate over the words in the sentence
    for idx, x in enumerate(probability_array):
        # Iterate over the words in the vocabulary
        for idy, y in enumerate(x):
            # Calculate the probability based on the edit distance between the sentence word and vocabulary word
            probability_array[idx][idy] = 1 / (edit_distance(sentence.split()[idx], vocab[idy][0]) + 1)

    return probability_array

##### *Function for the beam search decoder*

In [16]:
def beam_search_decoder(predictions, top_k = 3):
    #start with an empty sequence with zero score
    output_sequences = [([], 0)]
    
    #looping through all the predictions
    for token_probs in predictions:
        new_sequences = []
        # print(token_probs)
        
        #append new tokens to old sequences and re-score
        for old_seq, old_score in output_sequences:
            # print('old_seq',old_seq)
            # print('old_score',old_score)
            for char_index in range(len(token_probs)):
                # print('char_index',char_index)
                new_seq = old_seq + [char_index]
                # print('new_seq',new_seq)
                #considering log-likelihood for scoring
                new_score = old_score + math.log(token_probs[char_index])
                # print('token_probs[char_index]',token_probs[char_index])
                # print('math_token_probs',math.log(token_probs[char_index]))
                # print('new_score',new_score)
                new_sequences.append((new_seq, new_score))
                # print('new_sequences',new_sequences)
                
        #sort all new sequences in the de-creasing order of their score
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        # print('output_sequences',output_sequences)
        
        #select top-k based on score 
        # *Note- best sequence is with the highest score
        output_sequences = output_sequences[:top_k]
        
    return output_sequences

##### *Function to get bigram probabilities*

In [17]:
def calculate_bigram_probabilities(sentences, alpha):
    for sentence in sentences:
        sum_prob2 = 0
        sent = ['<start>'] + sentence + ['<end>']
        # Iterate over the bigrams of the sentence
        for idx in range(1, len(sent)):
        # calculating the prob while considering the prob of preceding word occuring with the unigram counter
            bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] +alpha) / (unigram_counter[(sent[idx-1],)] + alpha*vocab_size)
            sum_prob2 += math.log2(bigram_prob)
    return sum_prob2

##### *Function to find the top-k most probable sentences*

In [30]:
def spell_correction(test_sentence ,distance_array, top_k, vocabulary):
    
    # get top-k beam search decoder sequences
    beam_decoder = beam_search_decoder(distance_array, top_k = top_k)
    
    # get words from the vocabulary
    beam_decoder_translated = [([vocabulary[k][0] for k in x[0]],x[1]) for x in beam_decoder]
    
    # construct sentences 
    beam_decoder_sentences = list(map(itemgetter(0), beam_decoder_translated))
    
    # get probabilities
    beam_decoder_lm_prob = [calculate_bigram_probabilities([sentence],alpha_bigram) for sentence in beam_decoder_sentences]
    
    # merge sentences and probabilities
    final_beam_decoder = [(beam_decoder_translated[i][0], beam_decoder_translated[i][1]+beam_decoder_lm_prob[i]) for i in range(0, len(beam_decoder_lm_prob))]
    
    # find out-of-vocabulary words
    words_oov = []
    for idx,i in enumerate(test_sentence.split()):
        if (list(map(itemgetter(0),vocabulary)).count(i) == 0):
            words_oov.append(test_sentence.split()[idx])
    
    # sort based on the probability
    final_beam_decoder_sorted = sorted(final_beam_decoder, key = lambda val: val[1], reverse = True)
    
    # print initial sentence
    print('Test sentence:              ',test_sentence)
    print('Words not in vocabulary:    ', ' '.join(map(str,words_oov)))
    print('\n')
    print('='*100)
    print('\n')
    
    # print each possible sentence
    for i in range(top_k):
    
        print('Possible Sentence Number ', i+1 ,': ', " ".join(map(str,final_beam_decoder_sorted[i][0])))
        print('Score: {:.4f}'.format(final_beam_decoder_sorted[i][1]))
        print('\n')
    
    return words_oov, final_beam_decoder_sorted

In [31]:
# define test sentence 
test_sentence = 'Today I realy saw a milion people.'#'Today I saw a milion people realy.'

#calculate distance
edit_distance_array = calculate_normalized_edit_distance(test_sentence,vocabulary)

# spell correction
words_oov, alternative_sentences = spell_correction(test_sentence,edit_distance_array,5,vocabulary)

Test sentence:               Today I realy saw a milion people.
Words not in vocabulary:     Today realy saw milion people.




Possible Sentence Number  1 :  today I really law a million people
Score: -91.5305


Possible Sentence Number  2 :  today I real law a million people
Score: -91.6133


Possible Sentence Number  3 :  today I really say a million people
Score: -91.9945


Possible Sentence Number  4 :  today I real say a million people
Score: -92.0773


Possible Sentence Number  5 :  today I real say a billion people
Score: -96.8213


